In [1]:
import pandas as pd

# Load main dataset
df = pd.read_csv("diabetic_data.csv")

# Load mapping file
mapping_raw = pd.read_csv("IDS_mapping.csv")

print("Main Dataset Shape:", df.shape)
print("\nMain Dataset Columns:\n", df.columns)

print("\nMapping File Shape:", mapping_raw.shape)
print("\nMapping File Head:\n", mapping_raw.head(20))

Main Dataset Shape: (101766, 50)

Main Dataset Columns:
 Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

Mapping Fil

In [3]:
import pandas as pd

mapping_raw = pd.read_csv("IDS_mapping.csv")

# STEP 1B — Identify header rows
header_rows = [0]  # admission_type_id starts at row 0 (header row of file)
dynamic_headers = mapping_raw[
    mapping_raw.iloc[:,0].astype(str).str.contains("_id", na=False)
].index.tolist()

# Combine, dedupe, sort
header_rows += [idx for idx in dynamic_headers if idx != 0]
header_rows = sorted(header_rows)
header_rows.append(len(mapping_raw))  # end boundary

# Extract the sections
sections = []
for i in range(len(header_rows)-1):
    start = header_rows[i]
    end = header_rows[i+1]
    section = mapping_raw.iloc[start:end].reset_index(drop=True)
    sections.append(section)

# Rename the 3 parsed sections
admission_type_map = sections[0].rename(columns={
    'admission_type_id': 'id',
    'description': 'description'
})

discharge_disposition_map = sections[1].rename(columns={
    'discharge_disposition_id': 'id',
    'description': 'description'
})

admission_source_map = sections[2].rename(columns={
    'admission_source_id': 'id',
    'description': 'description'
})

# Display
print("Admission Type Map:", admission_type_map.shape)
print(admission_type_map.head(), "\n")

print("Discharge Disposition Map:", discharge_disposition_map.shape)
print(discharge_disposition_map.head(), "\n")

print("Admission Source Map:", admission_source_map.shape)
print(admission_source_map.head())

Admission Type Map: (9, 2)
  id    description
0  1      Emergency
1  2         Urgent
2  3       Elective
3  4        Newborn
4  5  Not Available 

Discharge Disposition Map: (32, 2)
          admission_type_id                                        description
0  discharge_disposition_id                                        description
1                         1                                 Discharged to home
2                         2  Discharged/transferred to another short term h...
3                         3                      Discharged/transferred to SNF
4                         4                      Discharged/transferred to ICF 

Admission Source Map: (26, 2)
     admission_type_id               description
0  admission_source_id               description
1                    1        Physician Referral
2                    2           Clinic Referral
3                    3              HMO Referral
4                    4  Transfer from a hospital


In [6]:
import pandas as pd
import numpy as np

# ----------------------------------------------------
# 0. FIX HEADERS FOR THE 2 MESSY LOOKUP TABLES
# ----------------------------------------------------

# Promote first row to header for discharge_disposition_map
dd_header = discharge_disposition_map.iloc[0]
discharge_disposition_map = discharge_disposition_map[1:].reset_index(drop=True)
discharge_disposition_map.columns = dd_header

# Promote first row to header for admission_source_map
as_header = admission_source_map.iloc[0]
admission_source_map = admission_source_map[1:].reset_index(drop=True)
admission_source_map.columns = as_header

# Now rename ID columns so "id" really exists
admission_type_map = admission_type_map.rename(columns={'admission_type_id': 'id'})
discharge_disposition_map = discharge_disposition_map.rename(columns={'discharge_disposition_id': 'id'})
admission_source_map = admission_source_map.rename(columns={'admission_source_id': 'id'})

# ----------------------------------------------------
# 1. Clean lookup tables
# ----------------------------------------------------

# Remove leftover header rows (now unnecessary)
discharge_disposition_map = discharge_disposition_map[
    discharge_disposition_map['id'] != "discharge_disposition_id"
].copy()

admission_source_map = admission_source_map[
    admission_source_map['id'] != "admission_source_id"
].copy()

# Convert IDs to numeric
admission_type_map['id'] = pd.to_numeric(admission_type_map['id'], errors='coerce')
discharge_disposition_map['id'] = pd.to_numeric(discharge_disposition_map['id'], errors='coerce')
admission_source_map['id'] = pd.to_numeric(admission_source_map['id'], errors='coerce')

# ----------------------------------------------------
# 2. Clean main dataset
# ----------------------------------------------------

df_clean = df.copy()

df_clean.replace("?", np.nan, inplace=True)

for col in ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

missing_summary = df_clean.isna().sum().sort_values(ascending=False)

print("Missing Values Summary:")
print(missing_summary.head(15), "\n")

print("Cleaned Main Dataset Preview:")
print(df_clean.head())

Missing Values Summary:
weight               98569
max_glu_serum        96420
A1Cresult            84748
medical_specialty    49949
payer_code           40256
race                  2273
diag_3                1423
diag_2                 358
diag_1                  21
encounter_id             0
troglitazone             0
tolbutamide              0
pioglitazone             0
rosiglitazone            0
acarbose                 0
dtype: int64 

Cleaned Main Dataset Preview:
   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)    NaN   
1        149190     55629189        Caucasian  Female  [10-20)    NaN   
2         64410     86047875  AfricanAmerican  Female  [20-30)    NaN   
3        500364     82442376        Caucasian    Male  [30-40)    NaN   
4         16680     42519267        Caucasian    Male  [40-50)    NaN   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0               

In [7]:
# -----------------------------
# STEP 1D — Merge lookup tables into main dataset
# -----------------------------

# Merge admission type descriptions
df_merged = df_clean.merge(
    admission_type_map,
    left_on='admission_type_id',
    right_on='id',
    how='left',
    suffixes=('', '_admtype')
).drop(columns=['id']).rename(columns={'description': 'admission_type_desc'})

# Merge discharge disposition descriptions
df_merged = df_merged.merge(
    discharge_disposition_map,
    left_on='discharge_disposition_id',
    right_on='id',
    how='left',
    suffixes=('', '_dischg')
).drop(columns=['id']).rename(columns={'description': 'discharge_disposition_desc'})

# Merge admission source descriptions
df_merged = df_merged.merge(
    admission_source_map,
    left_on='admission_source_id',
    right_on='id',
    how='left',
    suffixes=('', '_admsrc')
).drop(columns=['id']).rename(columns={'description': 'admission_source_desc'})

# Display preview and new columns
print("Merged Dataset Shape:", df_merged.shape)
print("\nNew Columns Added:")
print([c for c in df_merged.columns if c.endswith('_desc')])

print("\nPreview:")
print(df_merged[[
    'admission_type_id', 'admission_type_desc',
    'discharge_disposition_id', 'discharge_disposition_desc',
    'admission_source_id', 'admission_source_desc'
]].head())

Merged Dataset Shape: (101766, 53)

New Columns Added:
['admission_type_desc', 'discharge_disposition_desc', 'admission_source_desc']

Preview:
   admission_type_id admission_type_desc  discharge_disposition_id  \
0                  6                 NaN                        25   
1                  1           Emergency                         1   
2                  1           Emergency                         1   
3                  1           Emergency                         1   
4                  1           Emergency                         1   

  discharge_disposition_desc  admission_source_id admission_source_desc  
0                 Not Mapped                    1    Physician Referral  
1         Discharged to home                    7        Emergency Room  
2         Discharged to home                    7        Emergency Room  
3         Discharged to home                    7        Emergency Room  
4         Discharged to home                    7        Emergenc

In [8]:
# -----------------------------
# STEP 1E — Final cleaning before SQL export
# -----------------------------

df_sql = df_merged.copy()

# 1. Fill missing descriptions
df_sql['admission_type_desc'].fillna("Unknown", inplace=True)
df_sql['discharge_disposition_desc'].fillna("Unknown", inplace=True)
df_sql['admission_source_desc'].fillna("Unknown", inplace=True)

# 2. Standardize column names for SQL
df_sql.columns = (
    df_sql.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

# 3. Optional: sort dataset for readability
df_sql.sort_values(by=['encounter_id'], inplace=True)

# 4. Export to CSV
output_path = "df_clean_for_sql.csv"
df_sql.to_csv(output_path, index=False)

print("SQL-ready dataset created:", output_path)
print("Final Shape:", df_sql.shape)

print("\nPreview:")
print(df_sql.head())

/var/folders/80/qns4qhrj56lg74phvxs3sj0r0000gn/T/ipykernel_47023/2091857397.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sql['admission_type_desc'].fillna("Unknown", inplace=True)
/var/folders/80/qns4qhrj56lg74phvxs3sj0r0000gn/T/ipykernel_47023/2091857397.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are settin

SQL-ready dataset created: df_clean_for_sql.csv
Final Shape: (101766, 53)

Preview:
    encounter_id  patient_nbr             race  gender       age weight  \
8          12522     48330783        Caucasian  Female   [80-90)    NaN   
9          15738     63555939        Caucasian  Female  [90-100)    NaN   
4          16680     42519267        Caucasian    Male   [40-50)    NaN   
10         28236     89869032  AfricanAmerican  Female   [40-50)    NaN   
5          35754     82637451        Caucasian    Male   [50-60)    NaN   

    admission_type_id  discharge_disposition_id  admission_source_id  \
8                   2                         1                    4   
9                   3                         3                    4   
4                   1                         1                    7   
10                  1                         1                    7   
5                   2                         1                    2   

    time_in_hospital  ... glipiz